In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests

url = "https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml"
response = requests.get(url)

# Save the file locally
with open("haarcascade_frontalface_default.xml", "wb") as file:
    file.write(response.content)

print("Download complete!")



Download complete!


In [4]:
import os
print(os.path.exists("haarcascade_frontalface_default.xml"))  # Should print True


True


In [5]:
import cv2

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

if face_cascade.empty():
    print("Failed to load cascade classifier!")
else:
    print("Cascade classifier loaded successfully!")


Cascade classifier loaded successfully!


# Opening the Web Cam and detecting the face

In [6]:
# Opening the live photo frontal face
import cv2

# Load the pre-trained Haar cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open webcam (0 for default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break

    # Convert frame to grayscale (Haar cascades work better in grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the output
    cv2.imshow('Face Detection', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


# Detecting Face and Back of the Head

In [ ]:
#frontal and backward version
import cv2

# Load the pre-trained Haar Cascade classifiers for face and upper body detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
upperbody_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # If frame is read correctly, ret is True
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break
    
    # Convert frame to grayscale (Haar Cascade works on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5,minSize=(30, 30) )

    # If no faces are detected, try detecting upper bodies
    if len(faces) == 0:
        upper_bodies = upperbody_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        for (x, y, w, h) in upper_bodies:
            # Draw a rectangle around the detected upper body (back of the head approximation)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, 'Back of Head Detected', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            # Save the image when back of the head is detected
            cv2.imwrite('back_of_head.png', frame)
    else:
        # Draw a rectangle around each detected face
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue rectangle for face
            cv2.putText(frame, 'Face Detected', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            # Save the image when a face is detected
            cv2.imwrite('face_detected.png', frame)

    # Display the frame with rectangles
    cv2.imshow('Face and Head Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

# Setting the threshold and check for overcrowding

In [ ]:
#setting threshold so that it will display the crowd
import cv2
# Load the pre-trained Haar Cascade classifiers for face and upper body detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
upperbody_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Set the threshold for overcrowding
crowd_threshold = 5  # You can adjust this limit as needed

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # If frame is read correctly, ret is True
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break
    
    # Convert frame to grayscale (Haar Cascade works on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    num_individuals = len(faces)  # Start counting the number of detected individuals with faces

    # If no faces are detected, try detecting upper bodies
    if num_individuals == 0:
        upper_bodies = upperbody_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        num_individuals += len(upper_bodies)  # Add the detected upper bodies to the count
        for (x, y, w, h) in upper_bodies:
            # Draw a rectangle around the detected upper body (back of the head approximation)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, 'Back of Head Detected', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            # Save the image when back of the head is detected
            cv2.imwrite('back_of_head.png', frame)
    else:
        # Draw a rectangle around each detected face
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue rectangle for face
            cv2.putText(frame, 'Face Detected', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            # Save the image when a face is detected
            cv2.imwrite('face_detected.png', frame)

    # Check if the count exceeds the threshold and display overcrowding warning
    if num_individuals > crowd_threshold:
        cv2.putText(frame, 'Warning: Overcrowded!', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(frame, f'Occupancy: {num_individuals}/{crowd_threshold}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame with rectangles
    cv2.imshow('Face and Head Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


# Generating Voice Alert

In [ ]:
#voice assistant feature
import cv2
import pyttsx3

# Load the pre-trained Haar Cascade classifiers for face and upper body detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
upperbody_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Set the threshold for overcrowding
crowd_threshold = 5  # You can adjust this limit as needed
last_announcement = ""  # Track the last announcement to avoid repetitive announcements

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # If frame is read correctly, ret is True
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break
    
    # Convert frame to grayscale (Haar Cascade works on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    num_individuals = len(faces)  # Start counting the number of detected individuals with faces

    # If no faces are detected, try detecting upper bodies
    if num_individuals == 0:
        upper_bodies = upperbody_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        num_individuals += len(upper_bodies)  # Add the detected upper bodies to the count
        for (x, y, w, h) in upper_bodies:
            # Draw a rectangle around the detected upper body (back of the head approximation)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, 'Back of Head Detected', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            # Save the image when back of the head is detected
            cv2.imwrite('back_of_head.png', frame)
    else:
        # Draw a rectangle around each detected face
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue rectangle for face
            cv2.putText(frame, 'Face Detected', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            # Save the image when a face is detected
            cv2.imwrite('face_detected.png', frame)

    # Check if the count exceeds the threshold and display overcrowding warning
    if num_individuals > crowd_threshold:
        cv2.putText(frame, 'Warning: Overcrowded!', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        announcement = f"Warning: Overcrowded! {num_individuals} individuals detected."
    else:
        cv2.putText(frame, f'Occupancy: {num_individuals}/{crowd_threshold}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        announcement = f"{num_individuals} individuals detected. The occupancy is within the safe limit."

    # Make the voice announcement only if it's different from the last announcement
    if announcement != last_announcement:
        engine.say(announcement)
        engine.runAndWait()
        last_announcement = announcement  # Update last announcement to avoid repetition

    # Display the frame with rectangles
    cv2.imshow('Face and Head Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

# Crowd detection using CSRNet Model

In [ ]:
pip install torch torchvision numpy scipy opencv-python matplotlib h5py


In [ ]:
pip install kaggle


In [ ]:
import os
print(os.getcwd())  # This prints your Jupyter Notebook's working directory


In [ ]:
import zipfile

zip_path = "shanghaitech.zip"  # Change this to your actual ZIP file name if different
extract_path = "shanghaitech"  # Folder to extract

# Extract the ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


In [ ]:
import os

dataset_path = "shanghaitech"  # Change if you extracted it elsewhere

# List all files in the dataset directory
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        print(os.path.join(root, file))


# Defining the CSRNet Model

In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
from PIL import Image

# Step 1: Define Paths
DATASET_PATH = "shanghaitech"  # Root dataset folder

# Step 2: Define the CSRNet Model
class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()
        # Use the first 23 layers of VGG16 as the frontend
        self.frontend = models.vgg16(pretrained=True).features[:23]
        # Define the backend with additional convolutional layers
        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, kernel_size=1)
        )
    
    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        return x

In [ ]:
# Step 3: Preprocess the Image and Generate Density Map
def load_image(img_path):
    image = Image.open(img_path).convert('RGB')
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

def generate_density_map(image_shape, points):
    density_map = np.zeros(image_shape[:2], dtype=np.float32)
    for point in points:
        x, y = int(point[0]), int(point[1])
        if 0 <= y < density_map.shape[0] and 0 <= x < density_map.shape[1]:
            density_map[y, x] = 1
    return gaussian_filter(density_map, sigma=15)

In [ ]:
# Step 4: Load the Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CSRNet().to(device)



In [ ]:
# Step 5: Define Training Function
def train_model(model, dataloader, num_epochs=10, learning_rate=1e-4):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    for epoch in range(num_epochs):
        for images, density_maps in dataloader:
            images, density_maps = images.to(device), density_maps.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, density_maps)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")
    print("Training complete!")

In [ ]:
# Step 6: Implement Crowd Estimation with Voice Alert
def estimate_crowd(img_path, model):
    model.eval()
    image = load_image(img_path).to(device)
    with torch.no_grad():
        output = model(image).cpu().numpy()[0, 0]
    estimated_count = np.sum(output)
    print(f"Estimated crowd count: {int(estimated_count)}")
    if estimated_count > 50:
        os.system("say 'High crowd density detected. Take necessary precautions.'")
    plt.imshow(output, cmap='jet')
    plt.colorbar()
    plt.show()

In [ ]:
import cv2
import torch
import numpy as np
import torchvision.transforms as transforms
import os
from PIL import Image
import matplotlib.pyplot as plt

# Load CSRNet Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CSRNet().to(device)
model.eval()  # Set model to evaluation mode

# Define image transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to estimate crowd from a webcam frame
def estimate_crowd_from_frame(frame):
    # Convert OpenCV frame to PIL Image
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Transform image for model input
    image = transform(image).unsqueeze(0).to(device)
    
    # Forward pass through the model
    with torch.no_grad():
        output = model(image).cpu().numpy()[0, 0]
    
    # Estimate crowd count
    estimated_count = np.sum(output)
    
    # Show density map (optional)
    plt.imshow(output, cmap='jet')
    plt.colorbar()
    plt.show()
    
    # Print estimated count
    print(f"Estimated Crowd Count: {int(estimated_count)}")
    
    # Voice alert for high crowd density
    if estimated_count > 50:
        os.system("say 'High crowd density detected. Take necessary precautions.'")

    return int(estimated_count)

# Initialize webcam
cap = cv2.VideoCapture(0)  # Use 0 for default webcam

while True:
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        break
    
    # Estimate crowd density
    count = estimate_crowd_from_frame(frame)
    
    # Display count on video frame
    cv2.putText(frame, f"Crowd Count: {count}", (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Show the video frame
    cv2.imshow("Live Crowd Detection", frame)
    
    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
